In [1]:
import hashlib
import pandas as pd
import numpy as np

df = pd.read_excel('data/clients_login.xlsx')
df2 = pd.read_excel('data/clients_data.xlsx')
df3 = pd.read_excel('data/products.xlsx')
df

,ID,First_Name,Last_Name,Login,Email,Password
0,1,Emily,Park,emily_parrk,empark@examplemail.lol,epark123
1,2,Stan,Levinson,stantheman,stanlev@examplemail.lol,st%195$lev
2,3,Sam,Porter,samporter,ssporter@examplemail.lol,portorino98
3,4,Mary,Smith,marysmith99,themarysmith@examplemail.lol,462smary
4,5,John,Steinfeld,steinfeld_john,stein_john@examplemail.lol,4jg8$%hi
...,...,...,...,...,...,...
865,866,Michael,Davis,mda39,mda1983@examplemail.lol,"mi<#_>.dav,.,*:"
866,867,Tanya,Davis,tadav91,tandavi2014@examplemail.lol,"t-*,!~da-$,,<"
867,868,James,Parker,jameparke,jpa1954@examplemail.lol,"jam$~*$-park.>.,*"
868,869,Tanya,Parker,tanpark,tanyparke@examplemail.lol,tan*:~:_park-#_!#2012


In [2]:
len(hashlib.sha256(b"Nobody inspects the spammish repetition").hexdigest())

64

In [3]:
class sql_table_creator:
    def __init__(self, table_name, frame, primary_key):
        self.name = table_name
        self.frame = frame
        self.pk = primary_key
        
    def type_selector(self, dtype):
        dtype = str(dtype)
        if str(dtype) == 'object':
            return 'VARCHAR(64)'
        if 'int' in dtype:
            return 'INTEGER'
        if 'float' in dtype:
            return 'DECIMAL(6,0)'
        
    def create_table(self):
        sql_command = f"CREATE TABLE IF NOT EXISTS '{self.name}' ("
        data_types = self.frame.dtypes
        for col in self.frame:
            name = col.upper()
            d_type = self.type_selector(data_types[col])
            nullable_status = 'NOT NULL' if 'ID'in col else 'DEFAULT NULL'
            line = f"\n\t'{name}' {d_type} {nullable_status},"
            sql_command += line
        sql_command+=f"\n\tPRIMARY KEY ('{self.pk}')\n);"
        return sql_command
        

In [4]:
table = sql_table_creator(table_name = 'clients_login', frame = df, primary_key = 'ID')
sql_command = table.create_table()
print(sql_command)


table = sql_table_creator(table_name = 'clients_info', frame = df2, primary_key = 'ID')
sql_command = table.create_table()
print(sql_command)

table = sql_table_creator(table_name = 'products', frame = df3, primary_key = 'SKU_ID')
sql_command = table.create_table()
print(sql_command)

CREATE TABLE IF NOT EXISTS 'clients_login' (
	'ID' INTEGER NOT NULL,
	'FIRST_NAME' VARCHAR(64) DEFAULT NULL,
	'LAST_NAME' VARCHAR(64) DEFAULT NULL,
	'LOGIN' VARCHAR(64) DEFAULT NULL,
	'EMAIL' VARCHAR(64) DEFAULT NULL,
	'PASSWORD' VARCHAR(64) DEFAULT NULL,
	PRIMARY KEY ('ID')
);
CREATE TABLE IF NOT EXISTS 'clients_info' (
	'ID' INTEGER NOT NULL,
	'COUNTRY' VARCHAR(64) DEFAULT NULL,
	'CITY' VARCHAR(64) DEFAULT NULL,
	'ADDRESS' VARCHAR(64) DEFAULT NULL,
	'POSTCODES' INTEGER DEFAULT NULL,
	'YEAR_OF_BIRTH' DECIMAL(6,0) DEFAULT NULL,
	'PHONE' INTEGER DEFAULT NULL,
	PRIMARY KEY ('ID')
);
CREATE TABLE IF NOT EXISTS 'products' (
	'SKU_ID' INTEGER NOT NULL,
	'SKU_NAME' VARCHAR(64) DEFAULT NULL,
	'PRICE' DECIMAL(6,0) DEFAULT NULL,
	'STOCK' INTEGER DEFAULT NULL,
	'VAT_ID' DECIMAL(6,0) NOT NULL,
	'CATEGORY_ID' INTEGER NOT NULL,
	'DESCRIPTION' VARCHAR(64) DEFAULT NULL,
	PRIMARY KEY ('SKU_ID')
);


In [25]:
class sql_table_filler:
    def __init__(self, table_name, frame):
        self.name = table_name
        self.frame = frame
        
    def fill_table(self):
        sql_command = f"INSERT INTO '{self.name}' ({', '.join(list(self.frame.columns))}) VALUES\n"
        for row in list(self.frame.itertuples(index=False, name=None)):
            sql_command += f"{str(row)},\n"
        return sql_command[:-2] + ';'

In [26]:
a = sql_table_filler('clients_info', df2)
print(a.fill_table())

INSERT INTO 'clients_info' (ID, Country, City, Address, Postcodes, Year_of_Birth, Phone) VALUES
(0, 'France', 'Paris', 'Rua de Sao Pedro, 8', 149802, 1972.0, 60142746982),
(1, 'Portugal', 'Braga', 'Rua de Sao Pedro, 96', 412310, 2004.0, 82679662660),
(2, 'Netherlands', 'Amsterdam', '1905 Street, 77', 162268, 1943.0, 87581163500),
(3, 'China', 'Pekin', 'Cherchil Avenue, 16', 220648, 1977.0, 67923425562),
(4, 'UK', 'London', 'Via Cavour, 14', 524375, 1965.0, 88700984358),
(5, 'Ukrain', 'Kiev', 'Via Cavour, 79', 528309, 1985.0, 64018043475),
(6, 'Germany', 'Berlin', 'Lenin street, 66', 303558, 1940.0, 46498131305),
(7, 'USA', 'New York', 'Invalidenstraße, 25', 245487, 1969.0, 67567365566),
(8, 'France', 'Paris', 'Wall Street, 89', 556168, 1952.0, 31346336655),
(9, 'Germany', 'Frankfurt', 'Rua de Sao Pedro, 71', 650543, 2006.0, 77826286162),
(10, 'Japan', 'Tokyo', 'Invalidenstraße, 81', 270372, 1968.0, 96349502438),
(11, 'Ukrain', 'Kiev', 'Jhonsons Square, 56', 380771, 1968.0, 86401805633)

In [11]:
list(df.itertuples(index=False, name=None))

[(1, 'Emily', 'Park', 'emily_parrk', 'empark@examplemail.lol', 'epark123'),
 (2,
  'Stan',
  'Levinson',
  'stantheman',
  'stanlev@examplemail.lol',
  'st%195$lev'),
 (3, 'Sam', 'Porter', 'samporter', 'ssporter@examplemail.lol', 'portorino98'),
 (4,
  'Mary',
  'Smith',
  'marysmith99',
  'themarysmith@examplemail.lol',
  '462smary'),
 (5,
  'John',
  'Steinfeld',
  'steinfeld_john',
  'stein_john@examplemail.lol',
  '4jg8$%hi'),
 (6,
  'George',
  'Stark',
  'george_stark',
  'georgestark96@examplemail.lol',
  'w3jis_9uu'),
 (7,
  'Samantha',
  'Hill',
  'sammyhill',
  'sammyhill@examplemail.lol',
  'overthehill94'),
 (8,
  'Sarah',
  'Gomez',
  'sarah_go',
  'sarahgogo@examplemail.lol',
  'saharasarah89'),
 (9,
  'James',
  'Stamos',
  'thestamos',
  'thestamosjames@examplemail.lol',
  'jas_#0gh'),
 (10, 'Devorah', 'Lee', 'devlee', 'devlee56@examplemail.lol', 'sd-3sv2%'),
 (11, 'Kimberly', 'Jones', 'kimjo', 'kimmyjo@examplemail.lol', 'kim1999'),
 (12,
  'Felicity',
  'Winston',
  'f

In [87]:
cities = {'Russia' : ['Moscow', 'Ekaterinburg', 'St.Petersburg'],
          'USA' : ['New York', 'Boston', 'Chicago', 'San Francisco'],
          'Portugal' : ['Lisbon', 'Porto', 'Braga'],
          'Germany' : ['Berlin', 'Hamburg', 'Munich', 'Frankfurt'],
          'France' : ['Paris', 'Marseille', 'Lyon'],
          'Spain' : ['Barcelona', 'Madrid', 'Vigo'],
          'Italy' : ['Rome']}


streets = {'Russia' : ['Lenin Street', '1905 Street', 'Mira Avenue', 'Svobody Street'],
           'USA' : ['Martin Luther King Drive', 'Ross Clark Circle', 'Wall Street', 'Washington Street'],
           'Portugal' : ['Avenida da Liberdade', 'Rua Augusta', 'Rua Cor de Rosa', 'Rua da Bica'],
           'Germany' : ['Bernauer Straße', 'Ebertstraße', 'Chausseestraße', 'Kurfürstendamm'],
           'France' : ['Rue des Barres', 'Rue des Rosiers', 'Rue Chanoinesse', 'Rue Cremieux'],
           'Spain' : ['Calle Huertas', 'Calle Mayor', 'Calle de Preciados', 'Paseo del Prado'],
           'Italy' : ["Campo de' Fiori", "Via della Conciliazione", "Via del Corso", "Via Margutta"]}

zipcodes = {'Moscow' : '125130',
            'Ekaterinburg' : '239032',
            'St.Petersburg': '635342',
            'New York' : '35004',
            'Boston' : '37806',
            'Chicago' : '56004',
            'San Francisco' : '14824',
            'Lisbon' : '145-616',
            'Porto' : '432-516',
            'Braga' : '421-361',
            'Berlin' : '76234',
            'Hamburg' : '25138',
            'Munich' : '27391',
            'Frankfurt' : '56234',
            'Paris' : '57234',
            'Marseille' : '32983',
            'Lyon' : '93274',
            'Barcelona' : '01004',
            'Madrid' : '06923',
            'Vigo' : '33724',
            'Rome' : '98168'
           }

m_names = ['Andrew', 'Vladislav', 'Pedro', 'Jhonny', 
           'David', 'Ricardo', 'Cristiano', 'Jack', 
           'Robert', 'Martin', 'Diego', 'Daniel',
           'Liam', 'Oliver', 'James', 'William']

my_names = ['Ttyana', 'Maria', 'Patricia', 'Ines', 
           'Marianna', 'Anna', 'Julia', 'Helen', 
           'Barbara', 'Babette', 'Jane', 'Olivia',
           'Emma', 'Charlotte', 'Sophia']

l_names = ['Castro', 'Abramov', 'Zurabov', 'Pereira', 'Silva', 
           'Santos', 'Oliveira', 'Motta', 'Smith', 'Williams', 
           'Brown', 'Lavigne', 'Monet', 'Blanchet', 
           'Müller', 'Schneider', 'Schmidt']



def clients_data_generation():
    def login_generator(name, last_name, year):
        include_year = np.random.choice([True, False])
        cut_year = np.random.choice([True, False])
        num_of_letters = np.random.randint(2, 6)
        login = name[:num_of_letters-1] + last_name[:num_of_letters]
        if include_year:
            if cut_year:
                login += year[2:]
            else:
                login += year
        return login.lower()

    def email_generator(name, last_name, year):
        nickname = login_generator(name, last_name, str(year)) + '@examplemail.lol'
        return nickname

    my_m_names = np.random.choice(m_names, 1000)
    my_ml_names = np.random.choice(l_names, 1000)

    my_f_names = np.random.choice(f_names, 1000)
    my_fl_names = np.random.choice(l_names, 1000)

    male_df = pd.DataFrame(data = {'First_Name' : my_m_names, 
                                   'Last_Name' : my_ml_names, 
                                   'Sex' : 'M'}).drop_duplicates()
    
    female_df = pd.DataFrame(data = {'First_Name' : my_f_names, 
                                     'Last_Name' : my_fl_names, 
                                     'Sex' : 'F'}).drop_duplicates()
    clients = pd.concat([male_df, female_df])

    clients['Birth_year'] = np.random.randint(1941, 2015, len(clients))
    clients['Phone_number'] = np.random.randint(3*10e+9, 10e+10, len(clients))
    clients['email'] = clients.apply(lambda x: email_generator(x['First_Name'], 
                                                               x['Last_Name'],
                                                               x['Birth_year']), axis = 1)
    clients = clients.sample(frac = 1).reset_index(drop = True)
    clients['ID'] = np.arange(1, len(clients)+1)
    clients = clients[['ID', 'First_Name', 'Last_Name', 'Sex', 'Birth_year', 'Phone_number', 'email']]
    return clients

def address_data_generation(clients):
    def give_addr_2_all_customers(clients):
        countries = np.random.choice(list(cities.keys()), len(clients))
        cur_cities, cur_streets, cur_zipcodes = list(), list(), list()

        for country in countries:
            cur_cities.append(np.random.choice(cities[country]))
            cur_streets.append(np.random.choice(streets[country]))

        for city in cur_cities:
            cur_zipcodes.append(zipcodes[city])

        df = pd.DataFrame(data = {'Country' : countries,
                                  'City' : cur_cities,
                                  'Street' : cur_streets,
                                  'PostCode' : cur_zipcodes,
                                  'CUSTMER_ID' : clients['ID']})
        return df
    df = give_addr_2_all_customers(clients)
    some_dublicates_df = give_addr_2_all_customers(clients.sample(frac=0.3))
    df = pd.concat([df, some_dublicates_df]).reset_index(drop=True)
    df['Building'] = np.random.randint(1, 200, len(df))
    df['Appartment_tmp'] = np.random.randint(1, 1000, len(df))
    df['Appartment_presence'] = np.random.choice([1,None], p=[0.8, 0.2], size=len(df))
    df['Appartment'] = df['Appartment_tmp'] * df['Appartment_presence']
    df = df.sample(frac=1).reset_index(drop=True)
    df['ID'] = np.arange(1, len(df)+1)
    df = df[['ID', 'Country', 'City', 'Street', 'Building', 'Appartment', 'PostCode', 'CUSTMER_ID']]
    return df

In [88]:
clients = clients_data_generation()
addresses = address_data_generation(clients)

In [89]:
addresses[addresses['Appartment'].isnull()]

,ID,Country,City,Street,Building,Appartment,PostCode,CUSTMER_ID
2,3,Russia,St.Petersburg,1905 Street,10,NaN,635342,12
3,4,Spain,Vigo,Paseo del Prado,53,NaN,33724,83
6,7,USA,Boston,Washington Street,41,NaN,37806,435
8,9,Italy,Rome,Via della Conciliazione,190,NaN,98168,70
22,23,USA,San Francisco,Ross Clark Circle,44,NaN,14824,412
...,...,...,...,...,...,...,...,...
549,550,Germany,Frankfurt,Kurfürstendamm,71,NaN,56234,174
555,556,France,Paris,Rue des Barres,167,NaN,57234,228
561,562,France,Lyon,Rue Cremieux,41,NaN,93274,14
568,569,USA,New York,Washington Street,44,NaN,35004,368


In [90]:
addresses[addresses['Appartment'].notnull()]

,ID,Country,City,Street,Building,Appartment,PostCode,CUSTMER_ID
0,1,Spain,Vigo,Calle de Preciados,99,486,33724,423
1,2,France,Lyon,Rue des Barres,191,11,93274,209
4,5,Italy,Rome,Via del Corso,53,653,98168,28
5,6,France,Lyon,Rue Cremieux,46,618,93274,16
7,8,USA,New York,Martin Luther King Drive,31,456,35004,210
...,...,...,...,...,...,...,...,...
584,585,Germany,Berlin,Kurfürstendamm,7,334,76234,303
585,586,Russia,Moscow,1905 Street,104,392,125130,219
586,587,France,Paris,Rue des Rosiers,95,865,57234,312
587,588,France,Paris,Rue des Rosiers,50,958,57234,421
